In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
import faiss
import json
import uvicorn
import tqdm as notebook_tqdm

In [7]:
file_path='./hotel_bookings.csv'
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.fillna(method='ffill', inplace=True)  # Handle missing values
    return df
load_data(file_path)

/tmp/ipykernel_7639/3964819970.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Handle missing values


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01-07-15
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01-07-15
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,02-07-15
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,02-07-15
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03-07-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,485.0,0,Transient,96.14,0,0,Check-Out,06-09-17
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,485.0,0,Transient,225.43,0,2,Check-Out,07-09-17
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,485.0,0,Transient,157.71,0,4,Check-Out,07-09-17
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,485.0,0,Transient,104.40,0,0,Check-Out,07-09-17


In [13]:
def compute_analytics(df):
    revenue_trend = df.groupby('date')['revenue'].sum()
    cancellation_rate = (df['cancelled'].sum() / len(df)) * 100
    geo_distribution = df['location'].value_counts()
    lead_time_distribution = df['lead_time'].describe()
    return {
        'revenue_trend': revenue_trend.to_dict(),
        'cancellation_rate': cancellation_rate,
        'geo_distribution': geo_distribution.to_dict(),
        'lead_time_distribution': lead_time_distribution.to_dict()
    }


In [15]:
class VectorDB:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = faiss.IndexFlatL2(384)
        self.data = []
    
    def add_data(self, texts):
        vectors = self.model.encode(texts)
        self.index.add(np.array(vectors))
        self.data.extend(texts)
    
    def search(self, query, top_k=1):
        query_vector = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_vector), top_k)
        return [self.data[idx] for idx in indices[0]]

# FastAPI Setup
app = FastAPI()
db = VectorDB()
data = load_data('hotel_bookings.csv')
db.add_data(data['hotel'].tolist())

/tmp/ipykernel_7639/3964819970.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Handle missing values


In [21]:
class QueryRequest(BaseModel):
    question: str

@app.post("/analytics")
def get_analytics():
    return compute_analytics(data)

@app.post("/ask")
def ask_question(request: QueryRequest):
    response = db.search(request.question)
    return {"answer": response}

# import asyncio
# import uvicorn

# if __name__ == "__main__":
#     try:
#         loop = asyncio.get_running_loop()
#         loop.create_task(uvicorn.run(app, host="0.0.0.0", port=8000))
#     except RuntimeError:
#         uvicorn.run(app, host="0.0.0.0", port=8000)



In [25]:
import asyncio
import uvicorn

if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
        loop.create_task(uvicorn.run(app, host="0.0.0.0", port=8000))
    except RuntimeError:
        uvicorn.run(app, host="0.0.0.0", port=8000)


RuntimeError: asyncio.run() cannot be called from a running event loop